# Overview and Purpose


In [1]:
##written by Kat Raissi 
#created on November 22, 2019
#last modified on June 22, 2020


In [2]:
##the following script was written to idenitfy duplicate company names found in a CRM
##more specifically, the script identifies issues of missing/added words and mispellings

##SPECIAL NOTE: several of the cells are repeated; in an effort to reduce redundacy, explanations and comments are only found with the first instance

In [3]:
##issues the script still fails to address:
##the script was used on a test company; able to identify all variations EXCEPT when the first word of the company is ommited

# Preliminary Actions and Measures

In [4]:
##imports necessary packages
import pandas as pd
from itertools import product, combinations
import numpy as np

In [5]:
##creates dataframe and uploads data
original=pd.DataFrame()
original=pd.read_csv("Examples.csv")
original.set_index('Company_ID')
original

,Company_ID,Company_Name,Company_Description
0,1,The Color Red,Fakeone
1,2,Color Red,Faketwo
2,3,The Color Rad,Fakethree
3,4,The Col Red,Fakefour
4,5,The Color Red Inc,Fakefive
5,6,The Color Red INC,Fakesix
6,7,The COLOR red,Fakeseven
7,8,The Red Color,Fakeeight
8,9,Th Red Color,Fakenine
9,10,The Red Col,Faketen


# Exploration 

In [6]:
##counts the number of rows in the dataframe
len(original.index)

13

In [7]:
##counts the number of words in a company name
count = original["Company_Name"].str.split().apply(len).value_counts()
count

3    9
4    2
2    2
Name: Company_Name, dtype: int64

In [8]:
##calculates the mean number of words in a company name using a weighted average
weighted_avg = np.average(count.index, weights=count)
weighted_avg

3.0

# Transformation

In [9]:
##splits every company name into separate columns based on spaces
##makes all values in the columns strings
df = original["Company_Name"].str.split(" ", expand = True) 
df = df.applymap(str)
df

,0,1,2,3
0,The,Color,Red,None
1,Color,Red,None,None
2,The,Color,Rad,None
3,The,Col,Red,None
4,The,Color,Red,Inc
5,The,Color,Red,INC
6,The,COLOR,red,None
7,The,Red,Color,None
8,Th,Red,Color,
9,The,Red,Col,None


In [10]:
##uses itertools to make every two column combination 
cc = list(combinations(df.columns,2))
df = pd.concat([df[c[1]] + df[c[0]] for c in cc], axis=1, keys=cc)
df

0                               1                     2
           1          2          3         2          3          3
0   ColorThe     RedThe    NoneThe  RedColor  NoneColor    NoneRed
1   RedColor  NoneColor  NoneColor   NoneRed    NoneRed   NoneNone
2   ColorThe     RadThe    NoneThe  RadColor  NoneColor    NoneRad
3     ColThe     RedThe    NoneThe    RedCol    NoneCol    NoneRed
4   ColorThe     RedThe     IncThe  RedColor   IncColor     IncRed
5   ColorThe     RedThe     INCThe  RedColor   INCColor     INCRed
6   COLORThe     redThe    NoneThe  redCOLOR  NoneCOLOR    Nonered
7     RedThe   ColorThe    NoneThe  ColorRed    NoneRed  NoneColor
8      RedTh    ColorTh         Th  ColorRed        Red      Color
9     RedThe     ColThe    NoneThe    ColRed    NoneRed    NoneCol
10  ColorThe     RedThe    NoneThe  RedColor  NoneColor    NoneRed
11  RedColor  NoneColor  NoneColor   NoneRed    NoneRed   NoneNone
12    ColThe     RadThe    NoneThe    RadCol    NoneCol    NoneRad

# Multi-column Search

In [11]:
##counts the duplicates in the dataframe
##all the column combinations didnot create dupes 
duplicates=df.duplicated().sum()
duplicates

2

In [12]:
##shows the duplicates previously found in the dataframe
firstround = df[df.duplicated(keep='first')]
firstround.to_excel('firstround.xlsx')

In [13]:
##drops the duplicates from the dataframe
df.drop_duplicates(keep=False,inplace=True) 

In [14]:
##counts the perceived duplicates when considering less columns 
dftest = df.iloc[:,0:2]
duplicates=dftest.duplicated().sum()
duplicates

1

In [15]:
##creates a new dataframe with the duplicates dropped 
####counts the perceived duplicates to check that it matches with previous dataframe
df1=pd.DataFrame()
df1=df[df.columns[0:2]]
df1=df1.copy(deep=True)
duplicates=df1.duplicated().sum()
duplicates

1

In [16]:
##the following cell repeat the previous steps for additional rounds of search 
secondround = df1[df1.duplicated(keep=False)]
secondround.to_excel('secondround.xlsx')
df1.drop_duplicates(keep=False,inplace=True)

In [17]:
##the following cell repeat the previous steps for additional rounds of search 
dftest = df1.iloc[:,0:1]
duplicates=dftest.duplicated().sum()
duplicates

2

In [18]:
##the following cell repeat the previous steps for additional rounds of search 
df2=pd.DataFrame()
df2=df1[df1.columns[0:1]]
df2=df2.copy(deep=True)
duplicates=df2.duplicated().sum()
duplicates

2

In [19]:
##the following cell repeat the previous steps for additional rounds of search 
thirdround = df2[df2.duplicated(keep=False)]
thirdround.to_excel(r'thirdround.xlsx')
df2.drop_duplicates(keep=False,inplace=True)

In [20]:
##demosntrates how many additional duplicates are left
##from the dataframe below, it can be seen that only the company and two additional duplicates remain 
df2.head()

,0
,1
2,ColorThe
6,COLORThe
8,RedTh
